In [1]:
!nvidia-smi

Wed Nov 10 10:55:09 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import os
import sys

def set_environment() -> None:
    IN_COLAB = 'google.colab' in sys.modules
    IN_KAGGLE = 'kaggle_web_client' in sys.modules
    LOCAL = not (IN_KAGGLE or IN_COLAB)
    print(f'IN_COLAB:{IN_COLAB}, IN_KAGGLE:{IN_KAGGLE}, LOCAL:{LOCAL}')

    if IN_COLAB:
        from google.colab import drive
        drive.mount('/content/drive')
        if not os.path.exists("/content/drive/MyDrive/PetFinder-my-Pawpularity-Contest/data/input"):
            os.chdir("/content/drive/MyDrive/jukiya/")
            !pip install --quiet kaggle
            !mkdir -p ~/.kaggle
            !cp kaggle.json ~/.kaggle/
            !chmod 600 /root/.kaggle/kaggle.json
            os.chdir("/content/drive/MyDrive/PetFinder-my-Pawpularity-Contest/data/")
            !mkdir input
            os.chdir("/content/drive/MyDrive/PetFinder-my-Pawpularity-Contest//data/input/")
            # !kaggle competitions download -c petfinder-pawpularity-score
        os.chdir("/content/drive/MyDrive/PetFinder-my-Pawpularity-Contest//")
        !pip install --quiet -r requirements.txt
        !pip uninstall --quiet -y albumentations
        !pip install --quiet albumentations==0.4.6
    if IN_KAGGLE:
        os.chdir("../input/PetFinder-my-Pawpularity-Contest")
    !pip install --quiet -q pytorch-lightning
    !pip install --quiet torch_optimizer==0.1.0
    !pip install --quiet timm
    !pip install --quiet grad-cam
    return IN_COLAB, IN_KAGGLE, LOCAL

In [3]:
IN_COLAB, IN_KAGGLE, LOCAL = set_environment()

IN_COLAB:True, IN_KAGGLE:False, LOCAL:False
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import cv2
from PIL import Image

import os
import gc
import sys
import json
import math
import random
import time
# from time import time
from datetime import datetime
from collections import Counter, defaultdict

import scipy as sp
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set(style="whitegrid")

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

from tqdm.auto import tqdm
import category_encoders as ce

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau
from torch.cuda.amp import GradScaler
from torch.cuda.amp import autocast

import timm

import albumentations as A
from albumentations.pytorch import ToTensor, ToTensorV2
from albumentations import ImageOnlyTransform

from pytorch_grad_cam.utils.image import show_cam_on_image
from pytorch_grad_cam import GradCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM

from transformers import AdamW
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup

import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)


import pytorch_lightning as pl
from pytorch_lightning import Callback, seed_everything
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import CSVLogger

#if CFG.apex:
#    from apex import amp

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [5]:
class Config:
    apex = False
    DEBUG = False
    print_freq = 150
    num_workers = 4
    size = 512
    batch_size = 32
    exp_name = "petfinder_baseline"
    model_name = "tf_efficientnet_b0_ns"
    scheduler = "CosineAnnealingLR"
    epochs = 20
    T_max = 3
    lr = 1e-4
    min_lr = 1e-6
    weight_decay = 1e-6
    gradient_accumulation_steps = 1
    max_grad_norm = 1000
    seed = 42
    target_size = 1
    target_col = "Pawpularity"
    n_folds = 5
    trn_fold = [0, 1, 2, 3, 4]
    train = True
    grad_cam = True

In [6]:
if IN_COLAB:
    os.chdir("/content/drive/MyDrive/PetFinder-my-Pawpularity-Contest/code/")
    from util.AverageMeter import AverageMeter, asMinutes, timeSince
    from util.logger import Logger
    from util.timer import Timer
    os.chdir("/content/drive/MyDrive/PetFinder-my-Pawpularity-Contest/")
    LOG = "./log/"
    SUBMISSION = "./data/submission/"
    os.chdir("./models/")
    !mkdir {Config.exp_name}
    os.chdir("/content/drive/MyDrive/PetFinder-my-Pawpularity-Contest/")
    OUTPUT = f"./models/{Config.exp_name}"
    logger = Logger(LOG, exp_name=Config.exp_name)

mkdir: cannot create directory ‘petfinder_baseline’: File exists


In [7]:
if IN_COLAB:
    train_df = pd.read_csv("./data/input/train.csv")
    test_df = pd.read_csv("./data/input/test.csv")

In [8]:
def get_train_file_path(image_id):
    if IN_COLAB:
        return "./data/input/train/{}.jpg".format(image_id)
    else:
        return "../input/petfinder-pawpularity-score/train/{}.jpg".format(image_id)

def get_test_file_path(image_id, is_colab=False):
    if IN_COLAB:
        return "./data/input/test/{}.jpg".format(image_id)
    else:
        return "../input/petfinder-pawpularity-score/test/{}.jpg".format(image_id)


train_df["file_path"] = train_df["Id"].apply(lambda x: get_train_file_path(x))
test_df["file_path"] = test_df["Id"].apply(lambda x: get_test_file_path(x))

In [9]:
def get_score(y_true, y_pred):
    score = mean_squared_error(y_true, y_pred, squared=False) # RMSE
    return score

def seed_everything(SEED):
    random.seed(SEED)
    os.environ["PYTHONHASHSEED"] = str(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.backends.cudnn.deterministic = True
seed_everything(Config.seed)

In [10]:
class TrainDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.file_names = df['file_path'].values
        self.labels = df[Config.target_col].values
        self.transform = transform
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file_path = self.file_names[idx]
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            image = self.transform(image=image)['image']
        # label = torch.tensor(self.labels[idx], dtype=torch.float32)
        label = torch.tensor(self.labels[idx]).float()
        # image = torch.tensor(image, dtype=torch.float32)
        return image, label

    
class GradCAMDataset(Dataset):
    def __init__(self, df):
        self.df = df
        self.image_ids = df['Id'].values
        self.file_names = df['file_path'].values
        self.labels = df[Config.target_col].values
        self.transform = get_transforms(data='valid')
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        image_id = self.image_ids[idx]
        file_path = self.file_names[idx]
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        vis_image = cv2.resize(image, (Config.size, Config.size)).copy()
        if self.transform:
            image = self.transform(image=image)['image']
        # label = torch.tensor(self.labels[idx], dtype=torch.float32)
        label = torch.tensor(self.labels[idx]).float()
        # image = torch.tensor(image, dtype=torch.float32)
        return image_id, image, vis_image, label

In [11]:
# plt.figure(figsize=(20, 20))
# row, col = 5, 5
# for i in range(row * col):
#     plt.subplot(col, row, i+1)
#     image = cv2.imread(train_df.loc[i, 'file_path'])
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     target = train_df.loc[i, 'Pawpularity']
#     plt.imshow(image)
#     plt.title(f"target: {target}")
# plt.show()

In [12]:
if Config.DEBUG:
    train_dataset = TrainDataset(train_df, transform=get_transforms(phase="train"))
    train_dataloader = DataLoader(train_dataset, batch_size=Config.batch_size, shuffle=True, num_workers=Config.num_workers, pin_memory=True, drop_last=True)
    for step, (image, label) in enumerate(train_dataloader):
        if step >= 1:
            break
        print(step)

In [13]:
def get_transforms(*, phase):
    if phase == "train":
        return A.Compose([
                          A.RandomResizedCrop(Config.size, Config.size, scale=(0.85, 1.0)),
                          A.Normalize(
                              mean=[0.485, 0.456, 0.406],
                              std=[0.229, 0.224, 0.225]
                          ),
                          ToTensorV2(),
        ])
    elif phase == "valid":
        return A.Compose([
                          A.Resize(Config.size, Config.size),
                          A.Normalize(
                              mean=[0.485, 0.456, 0.406],
                              std=[0.229, 0.224, 0.225]
                          ),
                          ToTensorV2(),
        ])

In [14]:
num_bins = int(np.floor(1 + np.log2(len(train_df))))
train_df["bins"] = pd.cut(train_df[Config.target_col], bins=num_bins, labels=False)

Fold = StratifiedKFold(n_splits=Config.n_folds, shuffle=True, random_state=Config.seed)
for n, (train_index, val_index) in enumerate(Fold.split(train_df, train_df["bins"])):
    train_df.loc[val_index, 'fold'] = int(n)
train_df['fold'] = train_df['fold'].astype("int16")

In [15]:
class CustomModel(nn.Module):
    def __init__(self, cfg, pretrained=False):
        super().__init__()
        self.cfg = cfg
        self.model = timm.create_model(self.cfg.model_name, pretrained=pretrained)
        self.n_features = self.model.classifier.in_features
        self.model.classifier = nn.Identity()
        self.fc = nn.Linear(self.n_features, self.cfg.target_size)

    def feature(self, image):
        feature = self.model(image)
        return feature
        
    def forward(self, image):
        feature = self.feature(image)
        output = self.fc(feature)
        return output

In [16]:
class RMSELoss(nn.Module):
    def __init__(self, eps=1e-6):
        super().__init__()
        self.mse = nn.MSELoss()
        self.eps = eps
    def forward(self, yhat, y):
        loss = torch.sqrt(self.mse(yhat, y) + self.eps)
        return loss

In [17]:
def train_fn(fold, train_loader, model, criterion, optimizer, epoch, scheduler, device):
    model.train()
    if Config.apex:
        scaler = GradScaler()
    losses = AverageMeter()
    start = end = time.time()
    global_step = 0
    for step, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        if Config.apex:
            with autocast():
                y_preds = model(images)
                loss = criterion(y_preds.view(-1), labels)
        else:
            y_preds = model(images)
            loss = criterion(y_preds.view(-1), labels)
        # record loss
        losses.update(loss.item(), batch_size)
        if Config.gradient_accumulation_steps > 1:
            loss = loss / Config.gradient_accumulation_steps
        if Config.apex:
            scaler.scale(loss).backward()
        else:
            loss.backward()
        del loss
        gc.collect()
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), Config.max_grad_norm)
        if (step + 1) % Config.gradient_accumulation_steps == 0:
            if Config.apex:
                scaler.step(optimizer)
                scaler.update()
            else:
                optimizer.step()
            optimizer.zero_grad()
            global_step += 1
        end = time.time()
        # if step % Config.print_freq == 0 or step == (len(train_loader)-1):
        #     print('Epoch: [{0}][{1}/{2}] '
        #           'Elapsed {remain:s} '
        #           'Loss: {loss.val:.4f}({loss.avg:.4f}) '
        #           'Grad: {grad_norm:.4f}  '
        #           'LR: {lr:.6f}  '
        #           .format(epoch+1, step, len(train_loader), 
        #                   remain=timeSince(start, float(step+1)/len(train_loader)),
        #                   loss=losses,
        #                   grad_norm=grad_norm,
        #                   lr=scheduler.get_lr()[0]))
        # wandb.log({f"[fold{fold}] loss": losses.val,
        #            f"[fold{fold}] lr": scheduler.get_lr()[0]})
    return losses.avg


def valid_fn(valid_loader, model, criterion, device):
    model.eval()
    losses = AverageMeter()
    preds = []
    start = end = time.time()
    for step, (images, labels) in enumerate(valid_loader):
        images = images.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        # compute loss
        with torch.no_grad():
            y_preds = model(images)
        loss = criterion(y_preds.view(-1), labels)
        losses.update(loss.item(), batch_size)
        # record accuracy
        preds.append(y_preds.to('cpu').numpy())
        if Config.gradient_accumulation_steps > 1:
            loss = loss / Config.gradient_accumulation_steps
        del loss 
        gc.collect()
        end = time.time()
        # if step % Config.print_freq == 0 or step == (len(valid_loader)-1):
        #     print('EVAL: [{0}/{1}] '
        #           'Elapsed {remain:s} '
        #           'Loss: {loss.val:.4f}({loss.avg:.4f}) '
        #           .format(step, len(valid_loader),
        #                   loss=losses,
        #                   remain=timeSince(start, float(step+1)/len(valid_loader))))
    predictions = np.concatenate(preds)
    return losses.avg, predictions

In [18]:
def get_grad_cam(model, device, x_tensor, img, label, plot=False):
    result = {"vis": None, "img": None, "pred": None, "label": None}
    with torch.no_grad():
        pred = model(x_tensor.unsqueeze(0).to(device))
    pred = np.concatenate(pred.to('cpu').numpy())[0]
    target_layer = model.model.conv_head
    cam = GradCAM(model=model, target_layer=target_layer, use_cuda=torch.cuda.is_available())
    output = cam(input_tensor=x_tensor.unsqueeze(0).to(device))
    try:
        vis = show_cam_on_image(img / 255., output[0])
    except:
        return result
    if plot:
        fig, axes = plt.subplots(figsize=(8, 8), ncols=2)
        axes[0].imshow(vis)
        axes[0].set_title(f"pred={pred:.4f}")
        axes[1].imshow(img)
        axes[1].set_title(f"target={label}")
        plt.show()
    result = {"vis": vis, "img": img, "pred": pred, "label": label}
    torch.cuda.empty_cache()
    return result

In [19]:
def train_loop(folds, fold):
    
    logger.info(f"========== fold: {fold} training ==========")

    # ====================================================
    # loader
    # ====================================================
    trn_idx = folds[folds['fold'] != fold].index
    val_idx = folds[folds['fold'] == fold].index

    train_folds = folds.loc[trn_idx].reset_index(drop=True)
    valid_folds = folds.loc[val_idx].reset_index(drop=True)
    valid_labels = valid_folds[Config.target_col].values

    train_dataset = TrainDataset(train_folds, transform=get_transforms(phase='train'))
    valid_dataset = TrainDataset(valid_folds, transform=get_transforms(phase='train'))

    train_loader = DataLoader(train_dataset,
                              batch_size=Config.batch_size, 
                              shuffle=True, 
                              num_workers=Config.num_workers, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset, 
                              batch_size=Config.batch_size * 2, 
                              shuffle=False, 
                              num_workers=Config.num_workers, pin_memory=True, drop_last=False)
    
    # ====================================================
    # scheduler 
    # ====================================================
    def get_scheduler(optimizer):
        if Config.scheduler=='ReduceLROnPlateau':
            scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=Config.factor, patience=Config.patience, verbose=True, eps=Config.eps)
        elif Config.scheduler=='CosineAnnealingLR':
            scheduler = CosineAnnealingLR(optimizer, T_max=Config.T_max, eta_min=Config.min_lr, last_epoch=-1)
        elif Config.scheduler=='CosineAnnealingWarmRestarts':
            scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=Config.T_0, T_mult=1, eta_min=Config.min_lr, last_epoch=-1)
        return scheduler

    # ====================================================
    # model & optimizer
    # ====================================================
    model = CustomModel(Config, pretrained=True)
    model.to(device)

    optimizer = AdamW(model.parameters(), lr=Config.lr, weight_decay=Config.weight_decay)
    scheduler = get_scheduler(optimizer)

    # ====================================================
    # loop
    # ====================================================
    criterion = RMSELoss()

    best_score = np.inf
    best_loss = np.inf
    
    for epoch in range(Config.epochs):
        
        start_time = time.time()
        
        # train
        avg_loss = train_fn(fold, train_loader, model, criterion, optimizer, epoch, scheduler, device)

        # eval
        avg_val_loss, preds = valid_fn(valid_loader, model, criterion, device)
        
        if isinstance(scheduler, ReduceLROnPlateau):
            scheduler.step(avg_val_loss)
        elif isinstance(scheduler, CosineAnnealingLR):
            scheduler.step()
        elif isinstance(scheduler, CosineAnnealingWarmRestarts):
            scheduler.step()

        # scoring
        score = get_score(valid_labels, preds)

        elapsed = time.time() - start_time

        logger.info(f'Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s, lr: {optimizer.param_groups[0]["lr"]:.5f}')
        logger.info(f'Epoch {epoch+1} - Score: {score:.4f}')
        # wandb.log({f"[fold{fold}] epoch": epoch+1, 
        #            f"[fold{fold}] avg_train_loss": avg_loss, 
        #            f"[fold{fold}] avg_val_loss": avg_val_loss,
        #            f"[fold{fold}] score": score})

        if score < best_score:
            best_score = score
            logger.info(f'Epoch {epoch+1} - Save Best Score: {best_score:.4f} Model')
            torch.save({'model': model.state_dict(), 
                        'preds': preds},
                        OUTPUT+f'/{Config.model_name}_fold{fold}_best.pth')
    
    valid_folds['preds'] = torch.load(OUTPUT+f'/{Config.model_name}_fold{fold}_best.pth', 
                                      map_location=torch.device('cpu'))['preds']

    return valid_folds

In [20]:
def main():

    """
    Prepare: 1.train 
    """

    def get_result(result_df):
        preds = result_df['preds'].values
        labels = result_df[Config.target_col].values
        score = get_score(labels, preds)
        logger.info(f'Score: {score:<.4f}')
    
    if Config.train:
        # train 
        oof_df = pd.DataFrame()
        for fold in range(Config.n_folds):
            if fold in Config.trn_fold:
                _oof_df = train_loop(train_df, fold)
                oof_df = pd.concat([oof_df, _oof_df])
                logger.info(f"========== fold: {fold} result ==========")
                get_result(_oof_df)
        # CV result
        logger.info(f"========== CV ==========")
        get_result(oof_df)
        # save result
        oof_df.to_csv(OUTPUT+'/oof_df.csv', index=False)
    
    if Config.grad_cam:
        # wandb_table = wandb.Table(columns=["Id", "Pawpularity", "pred", "image", "grad_cam_image"])
        for fold in range(Config.n_folds):
            if fold in Config.trn_fold:
                # load model
                model = CustomModel(Config, pretrained=False)
                state = torch.load(OUTPUT+f'/{Config.model_name}_fold{fold}_best.pth', 
                                   map_location=torch.device('cpu'))['model']
                model.load_state_dict(state)
                model.to(device)
                model.eval()
                # load oof
                oof = pd.read_csv(OUTPUT+'/oof_df.csv')
                oof['diff'] = abs(oof['Pawpularity'] - oof['preds'])
                oof = oof[oof['fold'] == fold].reset_index(drop=True)
                # grad-cam (oof ascending=False)
                count = 0
                oof = oof.sort_values('diff', ascending=False)
                valid_dataset = GradCAMDataset(oof)
                for i in range(len(valid_dataset)):
                    image_id, x_tensor, img, label = valid_dataset[i]
                    result = get_grad_cam(model, device, x_tensor, img, label, plot=True)
                    if result["vis"] is not None:
                        count += 1
                        # wandb_table.add_data(image_id, 
                        #                      result["label"], 
                        #                      result["pred"], 
                        #                      wandb.Image(result["img"]), 
                        #                      wandb.Image(result["vis"]))
                    if count >= 5:
                        break
                # grad-cam (oof ascending=True)
                count = 0
                oof = oof.sort_values('diff', ascending=True)
                valid_dataset = GradCAMDataset(oof)
                for i in range(len(valid_dataset)):
                    image_id, x_tensor, img, label = valid_dataset[i]
                    result = get_grad_cam(model, device, x_tensor, img, label, plot=True)
                    if result["vis"] is not None:
                        count += 1
                        # wandb_table.add_data(image_id, 
                        #                      result["label"], 
                        #                      result["pred"], 
                        #                      wandb.Image(result["img"]), 
                        #                      wandb.Image(result["vis"]))
                    if count >= 5:
                        break

In [21]:
if __name__ == '__main__':
    main()

[2021-11-10 10:55:33] - ========== fold: 0 training ==========
[2021-11-10 11:03:21] - Epoch 1 - avg_train_loss: 35.3026  avg_val_loss: 29.7779  time: 465s, lr: 0.00008
[2021-11-10 11:03:21] - Epoch 1 - Score: 29.8950
[2021-11-10 11:03:21] - Epoch 1 - Save Best Score: 29.8950 Model
[2021-11-10 11:06:02] - Epoch 2 - avg_train_loss: 25.2824  avg_val_loss: 22.4983  time: 160s, lr: 0.00003
[2021-11-10 11:06:02] - Epoch 2 - Score: 22.6233
[2021-11-10 11:06:02] - Epoch 2 - Save Best Score: 22.6233 Model
[2021-11-10 11:08:42] - Epoch 3 - avg_train_loss: 20.6523  avg_val_loss: 21.1937  time: 160s, lr: 0.00000
[2021-11-10 11:08:42] - Epoch 3 - Score: 21.3131
[2021-11-10 11:08:42] - Epoch 3 - Save Best Score: 21.3131 Model
[2021-11-10 11:11:22] - Epoch 4 - avg_train_loss: 19.2783  avg_val_loss: 21.5196  time: 160s, lr: 0.00003
[2021-11-10 11:11:22] - Epoch 4 - Score: 21.6384
[2021-11-10 11:14:02] - Epoch 5 - avg_train_loss: 18.6347  avg_val_loss: 20.5332  time: 160s, lr: 0.00008
[2021-11-10 11:1

FileNotFoundError: ignored

In [23]:
def get_result(result_df):
        preds = result_df['preds'].values
        labels = result_df[Config.target_col].values
        score = get_score(labels, preds)
        logger.info(f'Score: {score:<.4f}')
        
for fold in range(Config.n_folds):
    if fold in [4]:
        _oof_df = train_loop(train_df, fold)
        oof_df = pd.read_csv("./models/petfinder_baseline/oof_df.csv")
        oof_df = pd.concat([oof_df, _oof_df])
        logger.info(f"========== fold: {fold} result ==========")
        get_result(_oof_df)

[2021-11-10 14:42:30] - ========== fold: 4 training ==========
[2021-11-10 14:45:11] - Epoch 1 - avg_train_loss: 35.2183  avg_val_loss: 29.8289  time: 161s, lr: 0.00008
[2021-11-10 14:45:11] - Epoch 1 - Score: 29.9660
[2021-11-10 14:45:11] - Epoch 1 - Save Best Score: 29.9660 Model
[2021-11-10 14:47:52] - Epoch 2 - avg_train_loss: 25.0662  avg_val_loss: 21.1303  time: 161s, lr: 0.00003
[2021-11-10 14:47:52] - Epoch 2 - Score: 21.2750
[2021-11-10 14:47:52] - Epoch 2 - Save Best Score: 21.2750 Model
[2021-11-10 14:50:32] - Epoch 3 - avg_train_loss: 20.4974  avg_val_loss: 22.3192  time: 161s, lr: 0.00000
[2021-11-10 14:50:32] - Epoch 3 - Score: 22.4624
[2021-11-10 14:53:13] - Epoch 4 - avg_train_loss: 19.0380  avg_val_loss: 21.3143  time: 160s, lr: 0.00003
[2021-11-10 14:53:13] - Epoch 4 - Score: 21.4555
[2021-11-10 14:55:53] - Epoch 5 - avg_train_loss: 18.3141  avg_val_loss: 20.4920  time: 161s, lr: 0.00008
[2021-11-10 14:55:53] - Epoch 5 - Score: 20.6234
[2021-11-10 14:55:53] - Epoch 5 

In [24]:
get_result(oof_df)

[2021-11-10 15:39:02] - Score: 19.3128


In [25]:
oof_df.to_csv(OUTPUT + f"/oof_df.csv", index=False)